# Немного теории

Для создания рекомендаций используем коллаборативную фильтрацию. Почему:
- не нужно подготавливать фичи
- есть достаточно много данных, причем пользователей гораздо больше, чем фильмов
- данный способ является менее затратным с точки зрения перерасчета модели при добавлении новых объектов, так новые фильмы добавляются реже, чем пользователи

В item-based подходе пользователь user_А характеризуется объектами, которые он просмотрел или оценил. Для каждого такого объекта определяется N объектов-соседей, т.е. находятся N наиболее похожих объектов с точки зрения просмотров/оценок пользователей. Все объекты-соседи объединяются во множество из которого исключаются объекты, которые были уже просмотрены пользователем user_А ранее. Из остальных строится top-k рекомендаций. Таким образом, при item-based подходе в создании рекомендаций участвуют все пользователи, которым понравился тот или иной фильм из просмотренных/оцененных пользователем user_A.

# Загрузка библиотек

In [1]:
import numpy as np 
import pandas as pd 
import itertools
from tqdm.notebook import tqdm
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
import seaborn as sns
import scipy
import scipy.sparse as sparse
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares # документация https://benfred.github.io/implicit/api/models/cpu/als.html#
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import MaxAbsScaler, OneHotEncoder

In [2]:
USER_ID = 3    # пользователь 
ITEM_ID = 3587 # фильм "Железный человек"
N = 10 # топ N фильмов

In [3]:
# !pip install pymorphy2

In [ ]:
# https://ru.stackoverflow.com/questions/995616/

import os
import requests
from pathlib import Path
import nltk
from nltk import sent_tokenize, word_tokenize, regexp_tokenize
from nltk.corpus import stopwords
import pymorphy2
from collections import Counter


url_stopwords_ru = "https://raw.githubusercontent.com/stopwords-iso/stopwords-ru/master/stopwords-ru.txt"


def get_text(url, encoding='utf-8', to_lower=True):
    url = str(url)
    if url.startswith('http'):
        r = requests.get(url)
        if not r.ok:
            r.raise_for_status()
        return r.text.lower() if to_lower else r.text
    elif os.path.exists(url):
        with open(url, encoding=encoding) as f:
            return f.read().lower() if to_lower else f.read()
    else:
        raise Exception('parameter [url] can be either URL or a filename')


def normalize_tokens(tokens):
    morph = pymorphy2.MorphAnalyzer()
    return [morph.parse(tok)[0].normal_form for tok in tokens]


def remove_stopwords(tokens, stopwords=None, min_length=4):
    if not stopwords:
        return tokens
    stopwords = set(stopwords)
    tokens = [tok
              for tok in tokens
              if tok not in stopwords and len(tok) >= min_length]
    return tokens


def tokenize_n_lemmatize(
    text, stopwords=None, normalize=True, 
    regexp=r'(?u)\b\w{4,}\b'):
    words = [w for sent in sent_tokenize(text)
             for w in regexp_tokenize(sent, regexp)]
    if normalize:
        words = normalize_tokens(words)
    if stopwords:
        words = remove_stopwords(words, stopwords)
    return words

stopwords_ru = get_text(url_stopwords_ru).splitlines()

# Загрузка и подготовка данных

In [3]:
users_df = pd.read_csv('/kaggle/input/diplom/users_processed.csv')
items_df = pd.read_csv('/kaggle/input/diplom/items_processed.csv')
interactions_df = pd.read_csv('/kaggle/input/diplom/interactions_processed.csv', parse_dates=['last_watch_dt'])

In [4]:
users_df.head(5)

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,True
1,962099,age_18_24,income_20_40,М,False
2,1047345,age_45_54,income_40_60,Ж,False
3,721985,age_45_54,income_20_40,Ж,False
4,704055,age_35_44,income_60_90,Ж,False


In [5]:
items_df.head(5)

,item_id,content_type,title,title_orig,release_year,genres,countries,age_rating,studios,directors,actors,description,keywords
0,10711,film,поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",испания,16.0,studios_unknown,педро альмодовар,"адольфо фернандес, ана фернандес, дарио гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",сша,16.0,studios_unknown,скот армстронг,"адам палли, брайан хаски, дж.б. смув, джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",канада,16.0,studios_unknown,адам п. калтраро,"адриан холмс, даррен шалави, джерри вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",великобритания,16.0,studios_unknown,эндрю хэй,"александра риддлстон-барретт, джеральдин джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,все решает мгновение,None_title_orig,1978.0,"драмы, спорт, советские, мелодрамы",ссср,12.0,ленфильм,виктор садовский,"александр абдулов, александр демьяненко, алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."


In [6]:
interactions_df.head(5)

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


In [7]:
interactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5476251 entries, 0 to 5476250
Data columns (total 5 columns):
 #   Column         Dtype         
---  ------         -----         
 0   user_id        int64         
 1   item_id        int64         
 2   last_watch_dt  datetime64[ns]
 3   total_dur      int64         
 4   watched_pct    float64       
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 208.9 MB


In [8]:
interactions_df[['user_id', 'item_id']]  = interactions_df[['user_id', 'item_id']].astype('int32')
interactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5476251 entries, 0 to 5476250
Data columns (total 5 columns):
 #   Column         Dtype         
---  ------         -----         
 0   user_id        int32         
 1   item_id        int32         
 2   last_watch_dt  datetime64[ns]
 3   total_dur      int64         
 4   watched_pct    float64       
dtypes: datetime64[ns](1), float64(1), int32(2), int64(1)
memory usage: 167.1 MB


Добавим новый признак. Бинарный. Будем считать, что фильм понравился если просмотрено 90% и более.

In [9]:
interactions_df['rec'] = interactions_df['watched_pct'].apply(lambda x: 1 if x>90 else 0)
interactions_df['rec'] = interactions_df['rec'].astype(bool)
interactions_df.head(5)

,user_id,item_id,last_watch_dt,total_dur,watched_pct,rec
0,176549,9506,2021-05-11,4250,72.0,False
1,699317,1659,2021-05-29,8317,100.0,True
2,656683,7107,2021-05-09,10,0.0,False
3,864613,7638,2021-07-05,14483,100.0,True
4,964868,9506,2021-04-30,6725,100.0,True


In [10]:
interactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5476251 entries, 0 to 5476250
Data columns (total 6 columns):
 #   Column         Dtype         
---  ------         -----         
 0   user_id        int32         
 1   item_id        int32         
 2   last_watch_dt  datetime64[ns]
 3   total_dur      int64         
 4   watched_pct    float64       
 5   rec            bool          
dtypes: bool(1), datetime64[ns](1), float64(1), int32(2), int64(1)
memory usage: 172.3 MB


In [11]:
# для записи результатов
results = pd.DataFrame(columns = ['RecSys', 'Precision', 'MAP'])
results.head()

,RecSys,Precision,MAP


# Разделение на трейн и тест

In [12]:
# сортировка по дате
interactions_df.sort_values(by='last_watch_dt', inplace = True) 

In [13]:
# 70% - на обучение, 30% - на тестирование
train_data, test_data= np.split(interactions_df, [int(.7*len(interactions_df))])

In [14]:
train_data.shape, test_data.shape

((3833375, 6), (1642876, 6))

In [15]:
# кол-во пользователей в датасете для обучения
train_data.user_id.nunique()

734170

In [16]:
# кол-во пользователей в датасете для теста
test_data.user_id.nunique()

419370

# Метрика

In [17]:
def compute_metrics(test, recs, top_N):
    
    '''
    Функция для подсчета метрик. 
    '''
    
    result = {}

    # объединяем тестовый датасет с полученными рекомендациями 
    test_recs = test.set_index(['user_id', 'item_id']).join(recs.set_index(['user_id', 'item_id']))
    # и сортируем по rank
    test_recs = test_recs.sort_values(by=['user_id', 'rank'])
    # считаем кол-во просмотренных фильмов
    test_recs['users_item_count'] = test_recs.groupby(level='user_id')['rank'].transform(np.size)
    # считаем reciprocal_rank = 1 / rank
    test_recs['reciprocal_rank'] = (1 / test_recs['rank']).fillna(0)
    # считаем cumulative_rank
    test_recs['cumulative_rank'] = test_recs.groupby(level='user_id').cumcount() + 1
    # считаем cumulative_rank / rank
    test_recs['cumulative_rank'] = test_recs['cumulative_rank'] / test_recs['rank']
    # считаем кол-во уникальных пользователей
    users_count = test_recs.index.get_level_values('user_id').nunique()

    # подсчет метрик
    result[f'Precision@{top_N}'] = test_recs['rank'].count()/ top_N / users_count
    result[f'MAP@{top_N}'] = (test_recs['cumulative_rank'] / test_recs['users_item_count']).sum() / users_count
    
    return pd.Series(result)

In [18]:
def compute_metrics_(test, recs, top_N):
    
    '''
    Та же самая функция для подсчета метрик, только с выводом промежуточных результатов. 
    Используется для тестирования и отладки на нескольких пользователях.
    '''
    
    result = {}
    
    print('Тестовый датасет:')
    print(test[['user_id', 'item_id']])
    print()
    print('Датасет с рекомендациями:')
    print(recs)   
    print()

    # объединяем тестовый датасет с полученными рекомендациями 
    print('Объединенный датасет:')
    test_recs = test.set_index(['user_id', 'item_id']).join(recs.set_index(['user_id', 'item_id']))
    print(test_recs.drop(columns=['last_watch_dt', 'total_dur', 'watched_pct', 'rec']))
    print()

    
    # и сортируем по rank
    print('Ранжирование:')
    test_recs = test_recs.sort_values(by=['user_id', 'rank'])
    print(test_recs.drop(columns=['last_watch_dt', 'total_dur', 'watched_pct', 'rec']))
    print()

    # считаем кол-во просмотренных фильмов
    print('Подсчет кол-во просмотренных фильмов')
    test_recs['users_item_count'] = test_recs.groupby(level='user_id')['rank'].transform(np.size)
    print(test_recs.drop(columns=['last_watch_dt', 'total_dur', 'watched_pct', 'rec']))
    print()

    # считаем reciprocal_rank = 1 / rank
    print('reciprocal_rank = 1 / rank')
    test_recs['reciprocal_rank'] = (1 / test_recs['rank']).fillna(0)
    print(test_recs.drop(columns=['last_watch_dt', 'total_dur', 'watched_pct', 'rec']))
    print()

    # считаем cumulative_rank
    print('cumulative_rank')
    test_recs['cumulative_rank'] = test_recs.groupby(level='user_id').cumcount() + 1
    print(test_recs.drop(columns=['last_watch_dt', 'total_dur', 'watched_pct', 'rec']))
    print()

    # считаем cumulative_rank / rank
    print('cumulative_rank / rank')
    test_recs['cumulative_rank'] = test_recs['cumulative_rank'] / test_recs['rank']
    print(test_recs.drop(columns=['last_watch_dt', 'total_dur', 'watched_pct', 'rec']))
    print()
    
    # считаем кол-во уникальных пользователей
    print('Кол-во уникальных пользователей')
    users_count = test_recs.index.get_level_values('user_id').nunique()
    print(users_count)
    print()

    # подсчет метрик
    result[f'Precision@{top_N}'] = test_recs['rank'].count()/ top_N / users_count
    result[f'MAP@{top_N}'] = (test_recs['cumulative_rank'] / test_recs['users_item_count']).sum() / users_count
    
    return pd.Series(result)

# Рекомендации на основе популярности

Подготовим два вида рекомендаци на основе популярности (самые просматриваемые фильмы).

1. В зависимости от социальных параметров пользователя (пол, возраст, наличие детей). Их будем применять к "холодным" пользователям (то есть к тем пользователям, по которым у нас еще нет ни одной записи), но есть соц.информация.
2. Пользователь холодный и по нему нет никакой информации.

Руководствуемся логикой "раз нравится большинству, то и тебе понравится".

In [19]:
# отфильтруем и оставим только самые просматриваемые фильмы (время просмотра более 90%)
# и сразу смержим с данными пользователей
popular = test_data[test_data.watched_pct > 90].merge(users_df)
popular

,user_id,item_id,last_watch_dt,total_dur,watched_pct,rec,age,income,sex,kids_flg
0,934357,8980,2021-07-24,7269,100.0,True,age_35_44,income_60_90,Ж,True
1,934357,734,2021-07-25,11101,100.0,True,age_35_44,income_60_90,Ж,True
2,934357,7160,2021-07-28,6774,96.0,True,age_35_44,income_60_90,Ж,True
3,934357,10440,2021-08-04,48210,100.0,True,age_35_44,income_60_90,Ж,True
4,724098,4495,2021-07-24,5159,100.0,True,age_55_64,income_20_40,М,False
...,...,...,...,...,...,...,...,...,...,...
280525,321,8335,2021-08-22,6064,100.0,True,age_45_54,income_40_60,М,False
280526,1054778,8534,2021-08-22,11015,100.0,True,age_25_34,income_40_60,М,False
280527,82640,7448,2021-08-22,58916,100.0,True,age_35_44,income_40_60,Ж,True
280528,211042,15464,2021-08-22,7975,100.0,True,age_45_54,income_20_40,М,False


In [20]:
# сгруппируем по нужным нам признакам и подсчитаем кол-во записей
soc_pop = popular.merge(users_df).groupby(['age', 'sex', 'kids_flg', 'item_id']).size().to_frame().reset_index()
soc_pop

,age,sex,kids_flg,item_id,0
0,age_18_24,sex_unknown,False,111,3
1,age_18_24,sex_unknown,False,456,1
2,age_18_24,sex_unknown,False,512,2
3,age_18_24,sex_unknown,False,517,1
4,age_18_24,sex_unknown,False,734,1
...,...,...,...,...,...
49482,age_unknown,М,True,12179,1
49483,age_unknown,М,True,12501,1
49484,age_unknown,М,True,12527,1
49485,age_unknown,М,True,13865,3


In [21]:
# составим таблицу со всеми возможными вариантами социальных групп

soc_pop_rec = []
for age in soc_pop.age.unique():
    for kids_flg in soc_pop.kids_flg.unique():
        for sex in soc_pop.sex.unique():
            
            top_items = soc_pop[(soc_pop.age == age) & (soc_pop.kids_flg == kids_flg) & (soc_pop.sex == sex)]. \
            sort_values(0, ascending=False).head(10).item_id.values
            
            soc_pop_rec.append([age, sex, kids_flg, top_items])

soc_pop_rec = pd.DataFrame(soc_pop_rec, columns = ['age', 'sex', 'kids_flg', 'item_id'])
soc_pop_rec

,age,sex,kids_flg,item_id
0,age_18_24,sex_unknown,False,"[13865, 9728, 3784, 3734, 7829, 111, 14488, 11..."
1,age_18_24,Ж,False,"[9728, 3734, 13865, 15297, 4151, 3784, 10440, ..."
2,age_18_24,М,False,"[9728, 3734, 13865, 3784, 7793, 15297, 10440, ..."
3,age_18_24,sex_unknown,True,[]
4,age_18_24,Ж,True,"[9728, 3734, 15297, 13865, 3784, 10440, 4151, ..."
5,age_18_24,М,True,"[9728, 3734, 13865, 7626, 7793, 15297, 3784, 4..."
6,age_25_34,sex_unknown,False,"[9728, 13865, 3734, 142, 15297, 15362, 512, 78..."
7,age_25_34,Ж,False,"[9728, 3734, 15297, 13865, 10440, 3784, 7571, ..."
8,age_25_34,М,False,"[9728, 3734, 13865, 3784, 15297, 10440, 7793, ..."
9,age_25_34,sex_unknown,True,"[14317, 1183, 3182, 3925, 4432, 6044, 6278, 76..."


In [22]:
# пример рекомендаций на основе популярности среди 35-44-летних пользователей-мужчин с детьми
for i in soc_pop_rec[(soc_pop_rec.age=='age_35_44') & (soc_pop_rec.sex=='М') & (soc_pop_rec.kids_flg==True) ].item_id.array[0]:
    title = str(*items_df.loc[items_df.item_id==i, 'title'].values)
    genres = str(*items_df.loc[items_df.item_id==i, 'genres'].values)
    release_year = str(*items_df.loc[items_df.item_id==i, 'release_year'].values)

    print(title.ljust(30, ' '), genres.ljust(50, ' '), release_year)

гнев человеческий              боевики, триллеры                                  2021.0
девятаев                       драмы, военные, приключения                        2021.0
прабабушка легкого поведения   комедии                                            2021.0
маленький воин                 семейное, комедии                                  2020.0
рядовой чээрин                 военные                                            2020.0
клиника счастья                драмы, мелодрамы                                   2021.0
радиовспышка                   боевики, драмы, фантастика, триллеры               2019.0
хрустальный                    триллеры, детективы                                2021.0
белый снег                     драмы, спорт                                       2021.0
поступь хаоса                  боевики, фантастика, фэнтези, приключения          2021.0


Без группировки

In [23]:
# подсчитаем кол-во записей
pop = popular.merge(users_df).groupby(['item_id']).size().to_frame().reset_index()
pop_rec = pop.sort_values(0, ascending=False).head(N).item_id.values
pop_rec

array([ 9728, 13865,  3734, 15297,  3784, 10440,   512,  7793,   142,
        7571])

In [24]:
# пример универсальной рекомендации
for i in pop_rec:
    title = str(*items_df.loc[items_df.item_id==i, 'title'].values)
    genres = str(*items_df.loc[items_df.item_id==i, 'genres'].values)
    release_year = str(*items_df.loc[items_df.item_id==i, 'release_year'].values)

    print(title.ljust(30, ' '), genres.ljust(60, ' '), release_year)

гнев человеческий              боевики, триллеры                                            2021.0
девятаев                       драмы, военные, приключения                                  2021.0
прабабушка легкого поведения   комедии                                                      2021.0
клиника счастья                драмы, мелодрамы                                             2021.0
маленький воин                 семейное, комедии                                            2020.0
хрустальный                    триллеры, детективы                                          2021.0
рядовой чээрин                 военные                                                      2020.0
радиовспышка                   боевики, драмы, фантастика, триллеры                         2019.0
маша                           драмы, триллеры                                              2020.0
100% волк                      мультфильм, приключения, семейное, фэнтези, комедии          2020.0


# Матрица взаимодействий

Основой для коллаборативной системы является матрица взаимодействий, подготовим ее.

In [25]:
# items
# уникальные значения id фильма
items_set = set(train_data.item_id)
# словарь для хранения соответсвия id фильма номеру строки в матрице 
items_dict = {item: iditem for item, iditem in zip(items_set, range(len(items_set)))}
# словарь для обратного перехода от номера строки в матрице к id фильма
inv_items_map = {v: k for k, v in items_dict.items()}

print(f'Кол-во фильмов: {len(items_set)}')

Кол-во фильмов: 15058


In [26]:
# аналогично для users
users_set = set(train_data.user_id)
users_dict = {user: iduser for user, iduser in zip(users_set, range(len(users_set)))}
inv_users_map = {v: k for k, v in users_dict.items()}

print(f'Кол-во пользователей: {len(users_set)}')

Кол-во пользователей: 734170


In [27]:
print(f'Размер будущей матрицы: {len(users_set)} х {len(items_set)}')

Размер будущей матрицы: 734170 х 15058


In [28]:
# значения для подстановки в матрицу
values = train_data.rec.tolist()
print(f'Кол-во записей о просмотрах: {len(values)}')

Кол-во записей о просмотрах: 3833375


In [29]:
# строки матрицы - пользователи
users = train_data.user_id.map(users_dict)
users

1863186    714134
3224846    492093
2162720    664475
3551672    287637
1299803    338981
            ...  
1120017    339820
5254544    133341
3104757    102847
1329935    214543
3321294    184099
Name: user_id, Length: 3833375, dtype: int64

In [30]:
# столбцы матрицы - фильмы
items = train_data.item_id.map(items_dict)
items

1863186     2142
3224846     9034
2162720     6739
3551672      302
1299803     3767
           ...  
1120017     6166
5254544    13994
3104757     9785
1329935      452
3321294     7710
Name: item_id, Length: 3833375, dtype: int64

In [31]:
# матрица user_item
user_item = csr_matrix((values, (users, items)), shape=(len(users_set), len(items_set)))
user_item

<734170x15058 sparse matrix of type '<class 'numpy.bool_'>'
	with 3833375 stored elements in Compressed Sparse Row format>

# Коллаборативная фильтрация - AlternatingLeastSquares

In [32]:
# default: factors=64, regularization=0.01, alpha=1.0, iterations=15, calculate_training_loss=False, random_state=None
model = AlternatingLeastSquares(factors=50, 
                                regularization=0.05, 
                                iterations=30, 
                                random_state=10)
model.fit(user_item)

  0%|          | 0/30 [00:00<?, ?it/s]

Модель может рекомендовать фильмы для пользователя, а также предлагать похожие фильмы.

In [33]:
# проверим на случайном пользователе

# перевод в индекс матрицы
id_user_ = users_dict[USER_ID]
# рекомендации - индекс в матрице
ids, scores = model.recommend(id_user_, user_item[id_user_])
# перевод в id фильма
ids = np.vectorize(inv_items_map.get)(ids)
# посмотрим на названия
for i in ids:
    print(*items_df.loc[items_df.item_id==i, 'title'].values)

королевский гамбит
марсианин
гонка
убийство в восточном экспрессе
патруль: по законам улиц
без компромиссов
альфа
бойцовский клуб
джон уик 3
левша


In [34]:
# проверим похожие фильмы

print(items_df.loc[items_df.item_id==ITEM_ID, 'title'].values)
print('Похожие фильмы: ')
# перевод в индекс матрицы
item_id_ = items_dict[ITEM_ID]
# похожие фильмы
ids, scores = model.similar_items(item_id_)
# перевод в id фильма
ids = np.vectorize(inv_items_map.get)(ids)
# посмотрим на названия
for i, d in zip(ids,scores) :
    title = str(*items_df.loc[items_df.item_id==i, 'title'].values)
    print(title.ljust(40, '.'), 'score ', d)    

['железный человек']
Похожие фильмы: 
железный человек........................ score  1.0000001
железный человек 2...................... score  0.9979548
железный человек 3...................... score  0.9976278
первый мститель......................... score  0.9883371
тор..................................... score  0.9842844
первый мститель: другая война........... score  0.974221
человек-муравей......................... score  0.9709954
тор: царство тьмы....................... score  0.9695387
первый мститель: противостояние......... score  0.9487252
лондон убивает.......................... score  0.94860816


## Тестирование

In [35]:
def recommends_als(x):
    
    '''
    Функция для
    - перевода id пользователя в индекс матрицы 
    - получения рекомендаций по обученной модели или по популярности
    - и обратный перевод из индекса матрицы в id фильма
    '''
    
    if (x in train_data.user_id.values) and (len(train_data[(train_data.user_id==x) & (train_data.rec)]) > 4): #если пользователь теплый и посмотрел полностью более .. фильмов
    #if (x in train_data.user_id.values) and (len(train_data[train_data.user_id==x]) > 4): #если пользователь теплый и посмотрел более .. фильмов
        x = users_dict[x]
        rec = pd.Series(model.recommend(x, user_item[x])[0]).map(inv_items_map).to_list()    

    elif x in users_df.user_id.values: # если пользователь холодный, но есть соц. инфо 
        age = users_df[users_df.user_id==x].age.values[0]
        kids_flg = users_df[users_df.user_id==x].kids_flg.values[0]
        sex = users_df[users_df.user_id==x].sex.values[0]
        rec_soc = soc_pop_rec[(soc_pop_rec.age==age) & (soc_pop_rec.sex==sex) & (soc_pop_rec.kids_flg==kids_flg) ].item_id.array[0]
        if len(rec_soc) >= N: # и по этой группе есть 10 фильмов
            rec = rec_soc
        else: # если нет 10 фильмов в этой группе
            rec = pop_rec

    else: # если пользователь холодный и нет никакой информации
        rec = pop_rec

    return rec

Проверим на очень маленьком датесете, состоящем из двоих пользователей.

In [36]:
# тестовый датасет
test_data_ = test_data[(test_data.user_id == 890658) | (test_data.user_id == 56062)]
test_data_

,user_id,item_id,last_watch_dt,total_dur,watched_pct,rec
570282,56062,5330,2021-08-15,121,2.0,False
339197,56062,9728,2021-08-15,8205,100.0,True
1702774,56062,7793,2021-08-15,8,0.0,False
2104012,56062,3475,2021-08-17,24,0.0,False
3107361,56062,6868,2021-08-17,3,0.0,False
4333160,56062,4436,2021-08-17,1286,15.0,False
4536957,890658,3734,2021-08-19,6201,100.0,True
4003938,890658,9728,2021-08-19,6,0.0,False
1796575,890658,13475,2021-08-20,2965,54.0,False
1681859,890658,16438,2021-08-20,5205,98.0,True


In [37]:
# таблица для записи рекомендаций
recs_ = pd.DataFrame({'user_id': test_data_['user_id'].unique()})
recs_

,user_id
0,56062
1,890658


In [38]:
# запишем рекомендованные фильмы по обученной ранее модели
recs_['item_id'] = recs_.user_id.apply(recommends_als)
# развернем список рекомендованных фильмов в строки
recs_ = recs_.explode(column=['item_id'])
# проранжируем рекомендованные фильмы
recs_['rank'] = recs_.groupby('user_id').cumcount() + 1
# посмотрим что получилось
recs_.head(15)

,user_id,item_id,rank
0,56062,9728,1
0,56062,13865,2
0,56062,15297,3
0,56062,3734,4
0,56062,10440,5
0,56062,512,6
0,56062,3784,7
0,56062,142,8
0,56062,5434,9
0,56062,7793,10


In [39]:
# посчитаем метрику
compute_metrics_(test_data_, recs_, 10)

Тестовый датасет:
         user_id  item_id
570282     56062     5330
339197     56062     9728
1702774    56062     7793
2104012    56062     3475
3107361    56062     6868
4333160    56062     4436
4536957   890658     3734
4003938   890658     9728
1796575   890658    13475
1681859   890658    16438
984646    890658    11829
1850055    56062    14414
1928376   890658    15574
312050    890658    15362
3736465    56062     3961
578264     56062     4912
4730890    56062     9622
2968836    56062     6192
4250412   890658     2722
4009502   890658    15384
124405     56062     4310
1698627    56062     1204
910020     56062     3963
1396387   890658    10755
2915874    56062    15126
5250503   890658     1337
714056     56062     2301

Датасет с рекомендациями:
   user_id item_id  rank
0    56062    9728     1
0    56062   13865     2
0    56062   15297     3
0    56062    3734     4
0    56062   10440     5
0    56062     512     6
0    56062    3784     7
0    56062     142     8
0 

Precision@10    0.200000
MAP@10          0.113258
dtype: float64

Визульно удостоверились, что все работает корректно. Можно протестировать на всем тестовом датасете.

In [40]:
# таблица для записи рекомендаций
recs_als = pd.DataFrame({'user_id': test_data.user_id.unique()})
recs_als.head()

,user_id
0,1021592
1,754719
2,934357
3,925575
4,760804


In [41]:
%%time
recs_list_als = [recommends_als(x) for x in tqdm(recs_als.user_id.values)]

  0%|          | 0/419370 [00:00<?, ?it/s]

CPU times: user 1h 3min 33s, sys: 23 s, total: 1h 3min 56s
Wall time: 58min 37s


In [42]:
recs_list_als[:2]

[array([ 9728, 13865,  3734,   512, 15297,  3784, 10440,  7793,   142,
         7829]),
 array([ 9728, 13865,  3734, 15297,  3784, 10440,   512,  7793,   142,
         7571])]

In [43]:
# запишем рекомендованные фильмы по обученной ранее модели
recs_als['item_id'] = recs_list_als

# развернем список рекомендованных фильмов в строки
recs_als = recs_als.explode(column=['item_id'])
# проранжируем рекомендованные фильмы
recs_als['rank'] = recs_als.groupby('user_id').cumcount() + 1
# посмотрим что получилось
recs_als.head(15)

,user_id,item_id,rank
0,1021592,9728,1
0,1021592,13865,2
0,1021592,3734,3
0,1021592,512,4
0,1021592,15297,5
0,1021592,3784,6
0,1021592,10440,7
0,1021592,7793,8
0,1021592,142,9
0,1021592,7829,10


In [44]:
# посчитаем метрику
res = compute_metrics(test_data, recs_als, N)
res

Precision@10    0.062437
MAP@10          0.105993
dtype: float64

Precision@10    0.062437 \
MAP@10          0.105993 \
dtype: float64

In [45]:
results.loc[len(results.index)] = ['ALS + pops', res[0], res[1]]
results

,RecSys,Precision,MAP
0,ALS + pops,0.062437,0.105993


А что если взять и предлагать всем просто самые популярные фильмы

In [46]:
def recommends_pops(x):
    
    if x in users_df.user_id.values: # если есть соц. инфо
        age = users_df[users_df.user_id==x].age.values[0]
        kids_flg = users_df[users_df.user_id==x].kids_flg.values[0]
        sex = users_df[users_df.user_id==x].sex.values[0]
        rec = soc_pop_rec[(soc_pop_rec.age==age) & (soc_pop_rec.sex==sex) & (soc_pop_rec.kids_flg==kids_flg) ].item_id.array[0]
    else: # если нет информации
        rec = pop_rec

    return rec

In [47]:
# таблица для записи рекомендаций
recs_pops = pd.DataFrame({'user_id': test_data.user_id.unique()})
recs_pops.head()

,user_id
0,1021592
1,754719
2,934357
3,925575
4,760804


In [48]:
%%time
recs_list_pops = [recommends_pops(x) for x in tqdm(recs_pops.user_id.values)]

  0%|          | 0/419370 [00:00<?, ?it/s]

CPU times: user 23min 23s, sys: 4.28 s, total: 23min 27s
Wall time: 23min 26s


In [49]:
# запишем рекомендованные фильмы
recs_pops['item_id'] = recs_list_pops

# развернем список рекомендованных фильмов в строки
recs_pops = recs_pops.explode(column=['item_id'])
# проранжируем рекомендованные фильмы
recs_pops['rank'] = recs_pops.groupby('user_id').cumcount() + 1
# посмотрим что получилось
recs_pops.head(15)

,user_id,item_id,rank
0,1021592,9728,1
0,1021592,13865,2
0,1021592,3734,3
0,1021592,512,4
0,1021592,15297,5
0,1021592,3784,6
0,1021592,10440,7
0,1021592,7793,8
0,1021592,142,9
0,1021592,7829,10


In [50]:
# посчитаем метрику
res = compute_metrics(test_data, recs_pops, N)
res

Precision@10    0.065820
MAP@10          0.109398
dtype: float64

Precision@10    0.065820 \
MAP@10          0.109398 \
dtype: float64

In [51]:
results.loc[len(results.index)] = ['pops', res[0], res[1]]
results

,RecSys,Precision,MAP
0,ALS + pops,0.062437,0.105993
1,pops,0.065820,0.109398


# Рекомендации на основе содержания

Нужно подготовить фичи объктов.

In [34]:
# возьмем только те объекты, которые есть в train (они хранятся в items_set)
data_content = items_df[items_df['item_id'].isin(items_set)]
data_content.sort_values('item_id', inplace=True)
data_content.head(5)

/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,item_id,content_type,title,title_orig,release_year,genres,countries,age_rating,studios,directors,actors,description,keywords
13315,0,series,смешарики: азбука безопасности,None_title_orig,2006.0,"русские, для детей, сериалы, хочу всё знать, р...",россия,0.0,studios_unknown,алексей горбунов,"антон виноградов, вадим бочанов",Опасности подстерегают на каждом шагу. И речь ...,"Смешарики, Азбука, безопасности, 2006, Россия,..."
1296,1,film,знаки,Signs,2002.0,"драмы, фантастика, триллеры, детективы",сша,12.0,studios_unknown,м. найт шьямалан,"мэл гибсон, хоакин феникс, рори калкин, эбигей...",Жизнь никогда не баловала фермера Грэма Хесса ...,"символизм, ферма, вера, инопланетянин, семейны..."
12711,2,film,жаркое по-французски,French roast,2008.0,"мультфильм, комедии",франция,6.0,studios_unknown,фабрис жубер,actors_unknown,Когда пришло время оплатить счёт в шикарном па...,"2008, франция, жаркое, по, французски"
14643,3,film,сонная лощина,Sleepy Hollow,1999.0,"ужасы, триллеры, детективы, фэнтези",сша,16.0,studios_unknown,тим бёртон,"джонни депп, кристина риччи, миранда ричардсон...","Нью-Йорк, 1799 год. Икабода Крэйна, молодого к...","ад, путешествие во времени, женщина-полицейски..."
4164,4,film,очень женские истории,None_title_orig,2020.0,"мелодрамы, русские, комедии",россия,18.0,studios_unknown,"анна саруханова, антон бильжо, лика ятковская,...","анастасия пронина, анна михалкова, анна слю, а...","Киноальманах из пяти короткометражных фильмов,...","Очень, женские, истории, 2020, Россия, друзья,..."


In [35]:
# items
# уникальные значения id фильма
items_set_content = set(data_content.item_id)
# словарь для хранения соответсвия id фильма номеру строки в матрице 
items_dict_content = {item: iditem for item, iditem in zip(items_set_content, range(len(items_set_content)))}
# словарь для обратного перехода от номера строки в матрице к id фильма
inv_items_map_content = {v: k for k, v in items_dict_content.items()}

print(f'Кол-во фильмов: {len(items_set_content)}')

Кол-во фильмов: 15057


In [36]:
data_content.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15057 entries, 13315 to 12591
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   item_id       15057 non-null  int64  
 1   content_type  15057 non-null  object 
 2   title         15057 non-null  object 
 3   title_orig    15057 non-null  object 
 4   release_year  15057 non-null  object 
 5   genres        15057 non-null  object 
 6   countries     15057 non-null  object 
 7   age_rating    15057 non-null  float64
 8   studios       15057 non-null  object 
 9   directors     15057 non-null  object 
 10  actors        15057 non-null  object 
 11  description   15057 non-null  object 
 12  keywords      15057 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 1.6+ MB


Объединим несколько признаков в один.

In [123]:
#data_content['content'] = data_content[['genres', 'countries', 'directors', 'actors', 'description']].agg(', '.join, axis=1)
#data_content.head(2)

In [124]:
#content = data_content['content'].values

In [43]:
#content[0]

In [44]:
#content = [tokenize_n_lemmatize(x, stopwords=stopwords_ru) for x in tqdm(content)]

In [45]:
#content[0]

In [46]:
#data_content['content'] = content

In [47]:
#data_content['content'].to_csv('content.csv', index=False)

In [48]:
#data_content['content'].to_csv('content_index.csv')

In [37]:
content = pd.read_csv('/kaggle/input/diplom/content.csv')
content

,content
0,"['ребёнок', 'сериал', 'мультфильм', 'развитие'..."
1,"['драма', 'фантастика', 'триллер', 'детектив',..."
2,"['мультфильм', 'комедия', 'франция', 'фабрис',..."
3,"['ужас', 'триллер', 'детектив', 'фэнтези', 'бё..."
4,"['мелодрама', 'комедия', 'анна', 'саруханов', ..."
...,...
15052,"['мелодрама', 'максим', 'субботина', 'алексей'..."
15053,"['криминал', 'мелодрама', 'виктор', 'сергеев',..."
15054,"['триллер', 'криминал', 'великобритания', 'аль..."
15055,"['драма', 'ссср', 'виктор', 'волков', 'алексей..."


In [38]:
# переведем эти признаки в векторы
tfidf = TfidfVectorizer() #max_features=500
X = tfidf.fit_transform(content.content) 
#X = CountVectorizer().fit_transform(items['content'])
X = MaxAbsScaler().fit_transform(X)
X

<15057x79887 sparse matrix of type '<class 'numpy.float64'>'
	with 919504 stored elements in Compressed Sparse Row format>

In [39]:
# найдем расстояния до соседей в этом признаком пространстве
neigh_items = NearestNeighbors(metric = 'cosine', algorithm = 'brute', n_neighbors=40)
neigh_items.fit(X)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=40)

In [40]:
# проверим похожие фильмы

print(items_df.loc[items_df.item_id==ITEM_ID, 'title'].values)
print('Похожие фильмы: ')
# перевод в индекс матрицы
item_id_ = items_dict_content[ITEM_ID]
# похожие фильмы
dists, ids = neigh_items.kneighbors(X[item_id_], 10)
# перевод в id фильма
ids = np.vectorize(inv_items_map.get)(ids)
# посмотрим на названия
for i, d in zip(*ids,*dists) :
    title = str(items_df.loc[items_df.item_id==i, 'title'].values)
    print(title.ljust(70, '.'), 'score ', d)   

['железный человек']
Похожие фильмы: 
['железный человек'].................................................. score  0.0
['железный человек 2']................................................ score  0.7276383214825564
['человек-паук: возвращение домой']................................... score  0.8209425896736033
['железный человек 3']................................................ score  0.8256363076102233
['план побега']....................................................... score  0.883161101635186
['мстители: финал']................................................... score  0.8942020359529614
['повар на колесах (жестовым языком)']................................ score  0.9139904874428508
['восстание']......................................................... score  0.9174916598834928
['мордекай'].......................................................... score  0.9179710187865464
['мстители'].......................................................... score  0.9182717867927455


In [41]:
def recommends_by_content(x):
    
    '''
    
   
    '''
    # фильмы, которые понравились пользователю
    liked_movies = train_data[(train_data.user_id==x) & (train_data.rec)].item_id.values
    
    if (x in train_data.user_id.values) and (len(liked_movies) > 0): #если пользователь теплый и есть понравившиеся фильмы
        #print('content')
        # списки для сохранения id фильмов и расстояний 
        rec_by_content_ids = []
        rec_by_content_dists = []

        for m in liked_movies:
            if m in items_df.item_id.values:
                # перевод в индекс матрицы
                item_id_ = items_dict_content[m]
                # похожие фильмы
                dists, ids = neigh_items.kneighbors(X[item_id_], N)
                # перевод в id фильма
                ids = np.vectorize(inv_items_map_content.get)(ids)
                #"раскроем скобки"
                ids = list(itertools.chain(*ids))
                dists = list(itertools.chain(*dists))  
                # запишем
                rec_by_content_ids.append(ids)
                rec_by_content_dists.append(dists)

        #"раскроем скобки"
        rec_by_content_ids = list(itertools.chain(*rec_by_content_ids))
        rec_by_content_dists = list(itertools.chain(*rec_by_content_dists))

        # уберем соседей с нулевым расстоянием (то есть сам фильм)
        rec_by_content = {}
        for i, d in zip(rec_by_content_ids, rec_by_content_dists):
            if d > 0.1:
                rec_by_content[i]=d

        # топ-N ближайших соседей        
        rec = list(dict(sorted(rec_by_content.items(), key=lambda item: item[1])).keys())[:N]
    
    elif x in users_df.user_id.values: # если пользователь холодный, но есть соц. инфо 
        #print('soc')
        age = users_df[users_df.user_id==x].age.values[0]
        kids_flg = users_df[users_df.user_id==x].kids_flg.values[0]
        sex = users_df[users_df.user_id==x].sex.values[0]
        rec_soc = soc_pop_rec[(soc_pop_rec.age==age) & (soc_pop_rec.sex==sex) & (soc_pop_rec.kids_flg==kids_flg) ].item_id.array[0]
        if len(rec_soc) >= N: # и по этой группе есть N фильмов
            rec = rec_soc
        else: # если нет N фильмов в этой группе
            rec = pop_rec

    else: # если пользователь холодный и нет никакой информации
        #print('pop')
        rec = pop_rec

    return rec

In [42]:
# тестовый датасет
test_data_ = test_data[(test_data.user_id == 1004246) | (test_data.user_id == 56062)]
test_data_

,user_id,item_id,last_watch_dt,total_dur,watched_pct,rec
1668333,1004246,11749,2021-07-24,714,11.0,False
2764668,1004246,11919,2021-07-24,857,13.0,False
4365552,1004246,3509,2021-07-26,864,15.0,False
702471,1004246,10605,2021-08-09,3023,54.0,False
3852174,1004246,799,2021-08-13,1740,33.0,False
570282,56062,5330,2021-08-15,121,2.0,False
339197,56062,9728,2021-08-15,8205,100.0,True
1702774,56062,7793,2021-08-15,8,0.0,False
2104012,56062,3475,2021-08-17,24,0.0,False
3107361,56062,6868,2021-08-17,3,0.0,False


In [43]:
# таблица для записи рекомендаций
recs_ = pd.DataFrame({'user_id': test_data_['user_id'].unique()})
recs_

,user_id
0,1004246
1,56062


In [44]:
# запишем рекомендованные фильмы по обученной ранее модели
recs_['item_id'] = recs_.user_id.apply(recommends_by_content)
# развернем список рекомендованных фильмов в строки
recs_ = recs_.explode(column=['item_id'])
# проранжируем рекомендованные фильмы
recs_['rank'] = recs_.groupby('user_id').cumcount() + 1
# посмотрим что получилось
recs_.head(15)

,user_id,item_id,rank
0,1004246,14250,1
0,1004246,16346,2
0,1004246,4273,3
0,1004246,6656,4
0,1004246,13073,5
0,1004246,5482,6
0,1004246,4445,7
0,1004246,6235,8
1,56062,9728,1
1,56062,13865,2


In [45]:
# посчитаем метрику
compute_metrics_(test_data_, recs_, N)

Тестовый датасет:
         user_id  item_id
1668333  1004246    11749
2764668  1004246    11919
4365552  1004246     3509
702471   1004246    10605
3852174  1004246      799
570282     56062     5330
339197     56062     9728
1702774    56062     7793
2104012    56062     3475
3107361    56062     6868
4333160    56062     4436
1850055    56062    14414
3736465    56062     3961
578264     56062     4912
4730890    56062     9622
2968836    56062     6192
124405     56062     4310
1698627    56062     1204
910020     56062     3963
2915874    56062    15126
714056     56062     2301

Датасет с рекомендациями:
   user_id item_id  rank
0  1004246   14250     1
0  1004246   16346     2
0  1004246    4273     3
0  1004246    6656     4
0  1004246   13073     5
0  1004246    5482     6
0  1004246    4445     7
0  1004246    6235     8
1    56062    9728     1
1    56062   13865     2
1    56062   15297     3
1    56062    3734     4
1    56062   10440     5
1    56062     512     6
1    560

Precision@10    0.1000
MAP@10          0.0375
dtype: float64

In [60]:
# таблица для записи рекомендаций
recs_content = pd.DataFrame({'user_id': test_data.user_id.unique()})
recs_content
# возьмем только часть тестовой выборки для уменьшения времени просчета
#recs_content = recs_content.head(500)

,user_id
0,1021592
1,754719
2,934357
3,925575
4,760804
...,...
419365,254206
419366,307468
419367,64089
419368,434000


Для ускорения распараллелим процесс \
https://python.plainenglish.io/simple-must-have-python-multiprocessing-90329dc0a0b8 \
https://github.com/dwei-cn/python-tricks/blob/main/Python_Multiprocessing_Medium_Dave_Wei.ipynb

In [61]:
import multiprocessing as mp
nprocs = mp.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 4


In [62]:
recs_content_list = list(recs_content.user_id.values)

In [63]:
# разделим список пользователей на количество процессоров (чанки)
def split(iteration, n):  
    k, m = divmod(len(iteration), n)
    split_data = [iteration[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n)]
    split_data_order_number = [[i, v] for i, v in enumerate(split_data)]
    return split_data_order_number
sub_list = split(recs_content_list, nprocs) 

In [64]:
%%time
import multiprocessing as mp
# initial Queue which will be used to save our output
# you can initiate as many Queue as you want
qout1 = mp.Queue()
qout2 = mp.Queue()
def func(x, q1, q2):
    '''
    define our function
    x: our input, a list containing sub lists 
    q1, q2: container for saving our result
    '''
    index = x[0]   # index of sub list
    value = x[1]   # content of each sub list
    res1 = []
    print(f'Job {index} starting\n')
    for i in tqdm(value):
      # here we are using calculating square value as example, you can replace it with any computation you want
      res1.append(list(recommends_by_content(i)))
      # in multiprocessing process，we use q.put() rather than return to save our output
    q1.put(res1)
    q2.put(index)
    print(f'Job {index} finishing\n')

CPU times: user 9.52 ms, sys: 2.97 ms, total: 12.5 ms
Wall time: 12.3 ms


In [65]:
%%time
processes = [mp.Process(target=func, args=(sub, qout1, qout2)) for sub in sub_list]
for p in processes:
    p.daemon = True
    p.start()
# the computation only get triggered when we run this q.get() method
unsorted_result = [[qout1.get(), qout2.get()] for p in processes]
# we can concatenate and sort result by using the index
result = sum([t[0] for t in sorted(unsorted_result, key=lambda x: x[1])], [])
for p in processes:
    # p.join tells the process to wait until all jobs finished then exit, effectively cleaning up the process.
    p.join()
# p.close terminate the process and tells the process not to accept any new job. 
    p.close()
print('All task is done!')

Job 0 starting

Job 1 starting

Job 2 starting

Job 3 starting



  0%|          | 0/104843 [00:00<?, ?it/s]

  0%|          | 0/104843 [00:00<?, ?it/s]

  0%|          | 0/104842 [00:00<?, ?it/s]

  0%|          | 0/104842 [00:00<?, ?it/s]

Job 3 finishing

Job 2 finishing

Job 1 finishing

Job 0 finishing

All task is done!
CPU times: user 5min 46s, sys: 2min 28s, total: 8min 14s
Wall time: 4h 24min 10s


In [85]:
#%%time
#recs_list_content = [recommends_by_content(x) for x in tqdm(recs_content.user_id.values)]

In [66]:
# запишем рекомендованные фильмы по обученной ранее модели
#recs_content['item_id'] = recs_list_content
recs_content['item_id'] = result

In [67]:
# развернем список рекомендованных фильмов в строки
recs_content = recs_content.explode(column=['item_id'])
# проранжируем рекомендованные фильмы
recs_content['rank'] = recs_content.groupby('user_id').cumcount() + 1
# посмотрим что получилось
recs_content.head(15)

,user_id,item_id,rank
0,1021592,9728,1
0,1021592,13865,2
0,1021592,3734,3
0,1021592,512,4
0,1021592,15297,5
0,1021592,3784,6
0,1021592,10440,7
0,1021592,7793,8
0,1021592,142,9
0,1021592,7829,10


In [68]:
recs_content.isna().sum()

user_id    0
item_id    8
rank       0
dtype: int64

In [69]:
# посчитаем метрику
res = compute_metrics(test_data, recs_content, N)
res

Precision@10    0.049589
MAP@10          0.089430
dtype: float64

Precision@10    0.049589 \
MAP@10          0.089430 \
dtype: float64

In [ ]:
results.loc[len(results.index)] = ['content + pops', res[0], res[1]]
results